In [ ]:
import midas.file_reader
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
import cygno as cy

mfile = cy.open_mid(run=1385, path='/tmp/', cloud=True, tag='LNGS', verbose=True)

for event in mfile:
    if event.header.is_midas_internal_event():
        print("Saw a special event")
        continue

    bank_names = ", ".join(b.name for b in event.banks.values())
    print("Event # %s of type ID %s contains banks %s" % (event.header.serial_number, event.header.event_id, bank_names))
    print("Received event with timestamp %s containing banks %s" % (event.header.timestamp, bank_names))
    print("%s, banks %s" % (datetime.utcfromtimestamp(event.header.timestamp).strftime('%Y-%m-%d %H:%M:%S'), bank_names))

    for bank_name, bank in event.banks.items():
        if bank_name=='DGH0': # PMTs wavform 
            header = cy.daq_dgz2header(bank)
            print (header)
            waveform = cy.daq_dgz2array(event.banks['DIG0'], header)
            plt.plot()
        if bank_name=='CAM0': # CAM image
            image, _, _ = cy.daq_cam2array(bank)
            plt.figure(figsize=(8,8))
            plt.imshow(image, vmin=95, vmax=150)
            plt.show()
        if bank_name=='INPT': # SLOW channels array
            slow = cy.daq_slow2array(bank)

In [ ]:
!rm /tmp/run01385.mid.gz

In [ ]:
import midas.file_reader
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
import cygno as cy

mfile = cy.open_mid(run=2300, path='/tmp/', cloud=True, tag='LNGS', verbose=True)

def plot_waveform(waveform, lenw, pmt, event_number, event_time):
    import numpy as np
    
    t = np.linspace(0,lenw, lenw)
    for ipmt in range(pmt):
        plt.subplot(pmt, 2, ipmt*2+2)
        plt.plot(t, waveform[ipmt])
        plt.show()
    return

for event in mfile:
    if event.header.is_midas_internal_event():
        print("Saw a special event")
        continue

    bank_names = ", ".join(b.name for b in event.banks.values())
    if event.header.serial_number%10 == 0:
        print("Event # %s of type ID %s contains banks %s" % (event.header.serial_number, event.header.event_id, bank_names))
        print("Received event with timestamp %s containing banks %s" % (event.header.timestamp, bank_names))
        print("%s, banks %s" % (datetime.utcfromtimestamp(event.header.timestamp).strftime('%Y-%m-%d %H:%M:%S'), bank_names))
        event_number = event.header.serial_number
        event_time = datetime.fromtimestamp(event.header.timestamp).strftime('%Y-%m-%d %H:%M:%S')
    for bank_name, bank in event.banks.items():
        if bank_name=='DGH0': # PMTs wavform 
            waveform_header = cy.daq_dgz2header(bank)
            print (waveform_header)
            waveform = cy.daq_dgz2array(event.banks['DIG0'], waveform_header)
            lenw = waveform_header[2]
            plot_waveform(waveform, lenw, 5, event_number, event_time)
        if bank_name=='CAM0': # CAM image
            image, _, _ = cy.daq_cam2array(bank)
#             plt.figure(figsize=(8,8))
#             plt.imshow(image, vmin=95, vmax=150)
#             plt.show()
        if bank_name=='INPT': # SLOW channels array
            slow = cy.daq_slow2array(bank)

In [ ]:
ls /tmp/his*

In [ ]:
# root access
#
import ROOT
f = ROOT.TFile.Open("/tmp/histograms_Run00089.root")
h = ROOT.TH2D("h1","Machine A + B",2304,0,2304,2304,0,2304);
h = f.Get('pic_run89_ev0')

c = ROOT.TCanvas();
#h.Draw()
#c.Draw()

In [ ]:
f.ls()

In [ ]:
h.RebinX(128)
h.RebinY(128)

h.Draw('col')
c.Draw()

In [ ]:
import cygno as cy

In [ ]:
cy.s3.obj_size('run01385.mid.gz', tag='LNGS', bucket='cygno-data', session="infncloud-iam", verbose=True)